In [ ]:
from jax.numpy import float64, zeros
from pandas import read_csv
from os import getcwd
from ticktack.fitting import SingleFitter

In [ ]:
fitting_object = SingleFitter(carbon_box_model, "Guttler14")

In [ ]:
def profile(func, args=[]) -> tuple:
    """
    Profiles the function, func, with the arguments, args, returning the speed and the variance of the speed.
    """
    time_sample = zeros(10)  # Storing the trials
    
    for i in range(10):
        timer = process_time()                          # starting a timer 
        func(*args).block_until_ready()                                     # Running the model 
        time_sample = time_sample.at[i].set(process_time() - timer)   # Stopping the timer and storing 

    return mean(time_sample), var(time_sample)

In [ ]:
results = {
    "module": [],
    "time": [],
    "type": []
}

odeints = {BS3 : "ode", DP5 : "jax", odeint_linear: "linear"} # For translation from string to module 

dataset = read_csv(f"{getcwd()}/datasets/775AD/NH/Miyake12_Cedar.csv") # Importing the miyake data

for namespace in odeints:# Iterating through the namespaces of odeint function 
    mean, variance = profile(carbon_box_model.run, [time_out, 100, \
                production, odeint[solver], y_initial, parameters, \
                PRODUCTION_RATE * UNIT_FACTOR]) )  # Profiling the speed
    results["module"].append(odeints[namespace])
    results["time"].append(float(mean) * 1000)
    results["type"].append("odeint")

    # Single variable function 
    loss = partial(loss_function, namespace.odeint, dataset, dydx, u0)

    # The gradients exist below 
    def gradient(p): 
        if namespaces[namespace] == "ode":
            return jacfwd(loss)(p)         # Calculating the gradient 
        else:
            return jacrev(loss)(p)

    mean, variance = profile(gradient, [parameters])

    results["module"].append(namespaces[namespace])
    results["time"].append(float(mean) * 1000)
    results["type"].append("gradient")

    def hessians(p): 
        if namespaces[namespace] == "ode":
            return jacfwd(jacfwd(loss))(p) # Calculating the hessian 
        else:
            return jacrev(jacrev(loss))(p)

    mean, variance = profile(hessians, [parameters])

    results["module"].append(namespaces[namespace])
    results["time"].append(float(mean) * 1000)
    results["type"].append("hessian")